<a href="https://colab.research.google.com/github/RajShah3006/Saarthi/blob/main/ai_university_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 google-generativeai gradio scikit-learn

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import google.generativeai as genai
from google.colab import userdata
from google.colab import drive # IMPORT DRIVE
import time
import concurrent.futures
import json
import os
import csv
import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# --- 1. SETUP & DRIVE MOUNT ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception:
    print("⚠️ API Key check failed. Ensure it is in secrets.")

# MOUNT GOOGLE DRIVE (The Permanent Storage)
print("📂 Connecting to Google Drive...")
drive.mount('/content/drive')

# Create a folder for your project in Drive if it doesn't exist
DRIVE_FOLDER = "/content/drive/My Drive/Saarthi_Project_Data"
if not os.path.exists(DRIVE_FOLDER):
    os.makedirs(DRIVE_FOLDER)

# Update paths to save inside Google Drive
CACHE_FILE = f"{DRIVE_FOLDER}/university_data_cached.json"
LOG_FILE = f"{DRIVE_FOLDER}/user_traffic_logs.csv"

print(f"✅ Data will be saved permanently to: {CACHE_FILE}")

model = genai.GenerativeModel('gemini-2.5-flash')
chat = model.start_chat(history=[])

# --- 2. LOGGING & CACHING ---

if not os.path.exists(LOG_FILE):
    with open(LOG_FILE, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Grade", "Location", "Interests", "Subjects"])

def log_interaction(grade, location, interests, subjects):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"\n🔔 [ALERT {timestamp}] New User from {location}!")
    with open(LOG_FILE, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([timestamp, grade, location, interests, subjects])

def save_data(data):
    with open(CACHE_FILE, 'w') as f:
        json.dump(data, f)
    print("💾 Database saved to Google Drive.")

def load_data():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as f:
            data = json.load(f)
            # Auto-Fix names if they are missing
            for p in data:
                if 'program_name' not in p and 'name' in p:
                    p['program_name'] = p['name']
            print(f"⚡ Loaded {len(data)} programs from Google Drive.")
            return data
    return None

# --- NEW: BATCH EMBEDDING (The Speed Fix) ---
def get_batch_embeddings(text_list):
    """
    Sends a list of texts to Google API in one go.
    Much faster than sending them one by one.
    """
    try:
        # Google text-embedding-004 supports batching
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=text_list,
            task_type="retrieval_document"
        )
        # The result['embedding'] will be a list of lists (vectors)
        return result['embedding']
    except Exception as e:
        print(f"Batch Error: {e}")
        # Fallback: return empty vectors if batch fails
        return [[0]*768 for _ in range(len(text_list))]

def get_single_embedding(text):
    # Keep this for the User Query (search)
    clean_text = re.sub(r'\s+', ' ', str(text)).strip()[:2000]
    try:
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=clean_text,
            task_type="retrieval_query" # Note: Query type for search
        )
        return result['embedding']
    except:
        return [0] * 768

# --- 3. SCRAPING FUNCTIONS ---
def list_all_programs(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        programs_list = []
        container = soup.select_one('div.results.results-programs')
        if not container: return None
        program_elements = container.select('h2.result-heading')
        for program_element in program_elements:
            program_name = program_element.get_text(strip=True)
            anchor_tag = program_element.find('a', href=True)
            if anchor_tag:
                programs_list.append({'name': program_name, 'url': anchor_tag['href']})
        return programs_list
    except:
        return None

def scrape_university_info(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        data = {}
        prerequisites = []
        headings = soup.find_all(string=re.compile(r'Prerequisites|Admission Requirements', re.IGNORECASE))
        for h in headings:
            lst = h.find_next(['ul', 'ol'])
            if lst: prerequisites.extend([li.get_text(strip=True) for li in lst.select('li')])
        if prerequisites: data['prerequisites'] = ", ".join(list(set(prerequisites)))
        avg = soup.find(string=re.compile(r'\d+%', re.IGNORECASE))
        if avg: data['admission_average'] = avg.strip()
        return data
    except:
        return {}

# --- 4. MAIN EXECUTION ---
all_programs_detailed_data = load_data()

if not all_programs_detailed_data:
    print("🚀 No cache found in Drive. Starting FRESH SCRAPE...")
    programs_with_urls = []
    alphabet_groups = ['a', 'b', 'c', 'd-e', 'f-g', 'h', 'i', 'j-l', 'm', 'n-p', 'q-s', 't-z']
    for group in alphabet_groups:
        res = list_all_programs(f"https://www.ouinfo.ca/programs/search/?search=&group={group}")
        if res: programs_with_urls.extend(res)
    print(f"✅ Found {len(programs_with_urls)} programs.")

    print("⏳ Scraping details (Parallel)...")
    scraped_results = []
    def process_program(entry):
        url = f"https://www.ouinfo.ca{entry['url']}"
        data = scrape_university_info(url)
        return {'program_name': entry['name'], 'program_url': url, **data}

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(process_program, p): p for p in programs_with_urls}
        completed = 0
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result: scraped_results.append(result)
            completed += 1
            if completed % 50 == 0: print(f"{completed}...", end="")

    # --- BATCH EMBEDDING LOGIC (The Fast Way) ---
    print("\n\n🧠 Generating AI Embeddings (BATCH MODE - 50x Faster)...")

    # 1. Prepare texts
    texts_to_embed = []
    for item in scraped_results:
        # Create the rich text string
        text = f"{item['program_name']} {item.get('prerequisites', '')}"
        # Clean it slightly
        text = re.sub(r'\s+', ' ', str(text)).strip()[:2000]
        texts_to_embed.append(text)

    # 2. Process in Batches of 50
    BATCH_SIZE = 50
    all_embeddings = []

    for i in range(0, len(texts_to_embed), BATCH_SIZE):
        batch = texts_to_embed[i : i + BATCH_SIZE]
        print(f"Processing batch {i} to {i+BATCH_SIZE}...", end="\r")

        batch_vectors = get_batch_embeddings(batch)
        all_embeddings.extend(batch_vectors)

        # Small sleep to be polite to the API rate limit
        time.sleep(1)

    # 3. Assign back to data
    all_programs_detailed_data = []
    for i, item in enumerate(scraped_results):
        item['embedding'] = all_embeddings[i]
        all_programs_detailed_data.append(item)

    save_data(all_programs_detailed_data)
    print("\n✅ Done! Data saved to Drive.")

# --- 5. CHATBOT & GRADIO ---
def find_best_matches(user_query, all_data, top_k=8):
    query_vector = get_single_embedding(user_query)
    valid_data = [p for p in all_data if 'embedding' in p]
    if not valid_data: return all_data[:5]
    db_vectors = [p['embedding'] for p in valid_data]
    scores = cosine_similarity([query_vector], db_vectors)[0]
    top_indices = scores.argsort()[-top_k:][::-1]
    return [valid_data[i] for i in top_indices]

def generate_chatbot_response(user_data, relevant_programs):
    context = ""
    for p in relevant_programs:
        name = p.get('program_name', 'Unknown Program')
        url = p.get('program_url', '#')
        avg = p.get('admission_average', 'Not Listed')
        prereqs = p.get('prerequisites', 'Not listed')
        context += f"- PROGRAM: {name}\n  AVG: {avg}\n  PREREQS: {prereqs}\n  LINK: {url}\n\n"

    prompt = f"""
    Act as 'Saarthi', a wise university guidance counselor in Ontario.

    STUDENT PROFILE:
    - Interest: {user_data['intrests']}
    - Grade: {user_data['grade']}
    - Avg: {user_data['overall_average']}
    - Subjects: {user_data['subjects']}
    - HOME LOCATION: {user_data['location']}

    TOP MATCHES:
    {context}

    INSTRUCTIONS:
    1. **Rank & Recommend:** Recommend the top 10 programs.
    2. **Prerequsite Check:** Compare "Subjects" vs "Prereqs". Warn if missing.
    3. **Fit Analysis:** Explain fit.
    4. **Extracurriculars:** Suggest side projects.
    5. **COMMUTE ANALYSIS:** - Calculate estimated travel time from '{user_data['location']}' to the university.
       - If > 1 hour, recommend RESIDENCE.
       - Estimate Cost (GO Train/Gas).
    6. **Tone:** Warm and supportive. Use emojis.
    """
    try:
        response = chat.send_message(prompt)
        return response.text
    except Exception as e:
        return f"Error: {e}"

def web_wrapper(subjects, interests, average, grade, location):
    log_interaction(grade, location, interests, subjects)

    user_data = {
        'subjects': subjects, 'intrests': interests,
        'overall_average': average, 'grade': grade, 'location': location
    }
    matches = find_best_matches(interests, all_programs_detailed_data)
    return generate_chatbot_response(user_data, matches)

interface = gr.Interface(
    fn=web_wrapper,
    inputs=[
        gr.Textbox(label="Current Subjects"),
        gr.Textbox(label="Interests"),
        gr.Textbox(label="Overall Average%"),
        gr.Textbox(label="Grade"),
        gr.Textbox(label="Location (City, ON)"),
    ],
    outputs=gr.Markdown(label="Saarthi Advice"),
    title="Saarthi AI: Commute & Program Guide",
)

interface.launch(inline=True, share=True, debug=True)

📂 Connecting to Google Drive...
Mounted at /content/drive
✅ Data will be saved permanently to: /content/drive/My Drive/Saarthi_Project_Data/university_data_cached.json
⚡ Loaded 1399 programs from Google Drive.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dfbb7001994c95c75e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🔔 [ALERT 2025-12-01 19:53:55] New User from Vaughan, ON!

🔔 [ALERT 2025-12-01 19:54:26] New User from Vaughan, ON!
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dfbb7001994c95c75e.gradio.live
